### Set Environment

In [51]:
import json
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### List all Output paths

In [37]:
from glob2 import glob
jsonFiles = glob('/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/*/posts/*.json') #Can be used absolute or relative paths

### Functiomn
* __Merge all data in files__
* __Normalizes merged .json into dataframe__
* __Saves as csv__

In [52]:
tiktok_data = pd.DataFrame([])

def merge_JsonFiles(filename):
    for f1 in filename:
        global tiktok_data
        with open(f1, 'r') as infile:
            data = json.load(infile)
        tiktok_data = tiktok_data.append(pd.json_normalize(data), ignore_index = True)
        tiktok_data.to_csv("/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/tiktok.csv", index=False)

merge_JsonFiles(jsonFiles)

# Data Summaries
## To Flourish: